# PSB injection bump in Xsuite - track!

In [1]:
%matplotlib notebook

In [2]:
# Import modules
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import xtrack as xt
import xpart as xp
import xdeps as xd

from cpymad.madx import Madx

/Users/giadarol/xsuite_packages/xtrack/xtrack/progress_indicator.py:90: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


###  Load line from previous notebook

In [3]:
# Reload
line = xt.Line.from_json('psb_05_thin.json')
line.build_tracker()
tw = line.twiss()

line.vars['t_turn_s'] = 0 # avoid leftovers from previous scripts

Loading line from dict:   0%|          | 0/2396 [00:00<?, ?it/s]

Done loading line from dict.           
Found suitable prebuilt kernel `default_only_xtrack`.
Found suitable prebuilt kernel `only_xtrack_frozen_energy`.


In [4]:
line.vars['volt_mv_h2'] = 0

### Enable chicane and chicane correction

In [5]:
line.vars['on_chicane_k0'] = 1
line.vars['on_chicane_k2'] = 1
line.vars['on_chicane_beta_corr'] = 1
line.vars['on_chicane_tune_corr'] = 1

### Install monitor at foil

In [6]:
monitor = xt.ParticlesMonitor(start_at_turn=0, stop_at_turn=6000, particle_id_range=(0,50))

line.discard_tracker()
line.insert_element(index='bi1.tstr1l1', element=monitor, name='monitor_at_foil')
line.build_tracker()

Found suitable prebuilt kernel `default_only_xtrack`.


### Generate some test particles

In [7]:
# p = xp.generate_matched_gaussian_bunch(num_particles=50, sigma_z=1e-3, nemitt_x=1e-8, nemitt_y=1.5e-8, line=line)

# # use the first particle as probe
# p.x[0] = 0
# p.px[0] = 0
# p.y[0] = 0
# p.py[0] = 0
# p.zeta[0] = 0
# p.delta[0] = 0

p = line.build_particles(zeta=np.linspace(0, 15, 50))

In [8]:
p.delta

LinkedArrayCpu([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
                0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
                0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
                0., 0., 0., 0., 0., 0., 0., 0.])

In [9]:
line.vv['t_turn_s'] = 0

In [10]:
line.track(p)

In [11]:
p.delta

LinkedArrayCpu([ 1.01429598e-09, -1.20075412e-07, -2.41147126e-07,
                -3.62182695e-07, -4.83163971e-07, -6.04072815e-07,
                -7.24891097e-07, -8.45600704e-07, -9.66183536e-07,
                -1.08662151e-06, -1.20689658e-06, -1.32699070e-06,
                -1.44688587e-06, -1.56656411e-06, -1.68600748e-06,
                -1.80519806e-06, -1.92411800e-06, -2.04274945e-06,
                -2.16107463e-06, -2.27907580e-06, -2.39673527e-06,
                -2.51403539e-06, -2.63095859e-06, -2.74748731e-06,
                -2.86360411e-06, -2.97929156e-06, -3.09453232e-06,
                -3.20930910e-06, -3.32360471e-06, -3.43740201e-06,
                -3.55068392e-06, -3.66343348e-06, -3.77563376e-06,
                -3.88726796e-06, -3.99831932e-06, -4.10877121e-06,
                -4.21860705e-06, -4.32781039e-06, -4.43636484e-06,
                -4.54425414e-06, -4.65146210e-06, -4.75797265e-06,
                -4.86376982e-06, -4.96883774e-06, -5.07316068e

In [12]:
monitor.delta[:, 0]

LinkedArrayCpu([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
                0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
                0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
                0., 0., 0., 0., 0., 0., 0., 0.])

### Enable time-dependent variables (update `t_turn_s` automatically during tracking)

In [ ]:
line.enable_time_dependent_vars = True
# line.dt_update_time_dependent_vars = 3e-6
line.vars.cache_active = True
line.vars['t_turn_s'] = 0.

In [ ]:
line.track(p)

In [ ]:
p.delta

In [ ]:
print('Tracking...')
line.track(p, num_turns=6000, time=True)
print(f'Done in {line.time_last_track:.4} s')

In [ ]:
import matplotlib.pyplot as plt
plt.close('all')
plt.figure(1)
plt.plot(monitor.x.T, alpha=0.5)
plt.plot(np.mean(monitor.x, axis=0), color='k')
plt.xlabel('Turn')
plt.ylabel('x [m]')
plt.show()

In [ ]:
plt.figure(2)
plt.plot(monitor.delta.T, alpha=0.5)
plt.xlabel('Turn')
plt.ylabel('delta')
plt.show()

In [ ]:
np.mean(monitor.x, axis=1)

In [ ]:
line.enable_time_dependent_vars = False

In [ ]:
tw = line.twiss(method='6d')

In [ ]:
plt.figure(100)
plt.plot(monitor.gamma0.T)

-------